In [20]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb

from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.datasets import load_iris, load_digits, load_boston

rng = np.random.RandomState(31337)

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 分割数据

In [3]:
% time rawdata_78 = pd.read_csv('rawdata_78.csv',index_col=1)

del rawdata_78['Unnamed: 0']
% time link_info = pd.read_csv('gy_contest_link_info.txt',sep=';') # 路段id,length,width,link_class,数据
link_ID_list = link_info['link_ID']

for i in link_ID_list:
    exec("ID_%s=rawdata_78.query(\"link_ID=='%s'\")" % (i,i))
    exec("ID_%s=ID_%s.copy()" % (i,i))

CPU times: user 756 ms, sys: 68 ms, total: 824 ms
Wall time: 822 ms
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.25 ms


# 清洗数据

In [4]:
def clean_2(tt):
    if shreshold[0]<tt<shreshold[1]:
        return tt
    elif tt>shreshold[1]:
        return shreshold[1]
    else:
        return shreshold[0]
for ID in link_ID_list:
    tt_describe = eval("ID_%s['travel_time'].describe()" % ID)
    tmp =tt_describe['75%']-tt_describe['25%']
    shreshold = [tt_describe['25%']-tmp*1.5, tt_describe['75%']+tmp*1.5]
    exec("ID_%s['travel_time_clean'] = ID_%s['travel_time'].apply(clean_2)" % (ID,ID))

# 制作link_top_info(每条link的物理特征)

In [5]:
link_top = pd.read_csv('gy_contest_link_top.txt',sep=';')# 路段拓扑关系数据
link_info = pd.read_csv('gy_contest_link_info.txt',sep=';') # 路段id,length,width,link_class,数据

In [6]:
def num_top(x):
    try:
        return len(x)
    except:
        return 0
link_top['in_num']=link_top['in_links'].str.split('#').apply(num_top)
link_top['out_num']=link_top['out_links'].str.split('#').apply(num_top)
link_top[['in0','in1','in2','in3']]=link_top['in_links'].str.split('#',expand=True) 
link_top[['out0','out1','out2','out3']]=link_top['out_links'].str.split('#',expand=True) 

In [7]:
def in_length(x):
    try:
        return link_info.query("link_ID=='%s'" % x)['length'].iloc[0]
    except:
        return 0
def in_width(x):
    try:
        return link_info.query("link_ID=='%s'" % x)['width'].iloc[0]
    except:
        return 0
for i in range(4):
    exec("link_top['in%d_len']=link_top['in%d'].apply(in_length)" % (i,i))
for i in range(4):
    exec("link_top['in%d_wid']=link_top['in%d'].apply(in_width)" % (i,i))
for i in range(4):
    exec("link_top['out%d_len']=link_top['out%d'].apply(in_length)" % (i,i))
for i in range(4):
    exec("link_top['out%d_wid']=link_top['out%d'].apply(in_width)" % (i,i))

In [8]:
link_top['in_width']=link_top['in0_wid']+link_top['in1_wid']+link_top['in2_wid']+link_top['in3_wid']
link_top['in_length']=(link_top['in0_len']+link_top['in1_len']+link_top['in2_len']+link_top['in3_len'])/link_top['in_num']
link_top['out_width']=link_top['out0_wid']+link_top['out1_wid']+link_top['out2_wid']+link_top['out3_wid']
link_top['out_length']=(link_top['out0_len']+link_top['out1_len']+link_top['out2_len']+link_top['out3_len'])/link_top['out_num']

In [9]:
length_mean=link_info['length'].mean()
link_top['in_length'].fillna(length_mean, inplace=True)
link_top['out_length'].fillna(length_mean, inplace=True)

In [10]:
del link_top['in_links']
del link_top['out_links']

In [11]:
link_info_top = pd.merge(link_info, link_top, how='left',on='link_ID')

# 把物理特征合并进入训练数据

In [12]:
for ID in link_ID_list:
    exec("ID_%s=pd.merge(ID_%s, link_info_top, how='left',on='link_ID')" % (ID, ID))

# 把时间特征合并近入训练数据

In [13]:
#当前所有时间特征:month, day, dayofweek, hour, minute, dayofyear, minuteofday, minuteofweek, hourofweek, hourofmonth
def link_to_datetime(ID_link):
    ID_link['time_interval_start']=pd.to_datetime(ID_link['time_interval_start'])
    ID_link['time_interval_end']=pd.to_datetime(ID_link['time_interval_end'])
    return ID_link
def link_datetime(ID_link):
    
    ID_link['minute']=ID_link['time_interval_start'].dt.minute
    ID_link['time_area']=ID_link['minute']//2
    ID_link['minuteofday']=ID_link['hour']*60+ID_link['minute']
    ID_link['dayofweek']=ID_link['time_interval_start'].dt.weekday
    ID_link['day']=ID_link['time_interval_start'].dt.day
    ID_link['month']=ID_link['time_interval_start'].dt.month
    ID_link['minuteofmonth']= ID_link['day']*24*60+ID_link['hour']*60+  ID_link['minute']
    ID_link['minuteofyear']=ID_link['month']*31*24*60+ID_link['day']*24*60+ID_link['hour']*60+  ID_link['minute']
    ID_link['dayofyear'] = ID_link['time_interval_start'].dt.dayofyear
    return ID_link

In [14]:
for ID in link_ID_list:
    exec("ID_%s=link_to_datetime(ID_%s)" % (ID,ID))
    exec("ID_%s=link_datetime(ID_%s)" % (ID,ID))

## 每条link的上一小时tt平均值特征:

In [15]:
def prehour_ttmean(ID_link):
    import numpy as np
    # 输入一条link的所有数据, 输出一条link的7点到8点的数据,其中数据列'prehour_ttmean'是6点到7点的travel_time平均值
    ID_link_hour7=ID_link.query("hour==7").copy()
    ID_link_hour8=ID_link.query("hour==8").copy()
    
    ID_link_hour8_prehourttmean=ID_link_hour7[['dayofyear','travel_time']].groupby('dayofyear').agg(np.mean)
    ID_link_hour8_prehourttmean.columns=['prehour_ttmean']
    ID_link_hour8_prehourttmean.reset_index(inplace=True)
    
    ID_link_hour8_more=pd.merge(ID_link_hour8, ID_link_hour8_prehourttmean, how='left',on='dayofyear')
    return ID_link_hour8_more

## 每条link的上游的上一小时tt平均值特征

In [16]:
import numpy as np
def inPreHourTTMean(ID_link):
    in_num = ID_link['in_num'].iloc[0]
    if in_num==1:
        in0_ID = ID_link['in0'].iloc[0]
        try:
            in0_hour6 = eval('ID_%s' % in0_ID).query('hour==7').copy()
        except:
            exec('ID_%s = link_datetime(ID_%s)' % (in0_ID,in0_ID))
            in0_hour6 = eval('ID_%s' % in0_ID).query('hour==7').copy()
        link_hour7 = ID_link.query('hour==8').copy()

        link_hour7_inPreHourTTMean = in0_hour6[['dayofyear','travel_time']].groupby('dayofyear').agg(np.mean)
        link_hour7_inPreHourTTMean.columns=['inPreHourTTMean']
        link_hour7_inPreHourTTMean.reset_index(inplace=True)

        link_hour7_more = pd.merge(link_hour7, link_hour7_inPreHourTTMean, how='left', on='dayofyear')
    elif in_num==2:
        in0_ID = ID_link['in0'].iloc[0]
        in1_ID = ID_link['in1'].iloc[0]
        try:
            in0_hour6 = eval('ID_%s' % in0_ID).query('hour==7').copy()
        except:
            exec('ID_%s = link_datetime(ID_%s)' % (in0_ID,in0_ID))
            in0_hour6 = eval('ID_%s' % in0_ID).query('hour==7').copy()
        try:
            in1_hour6 = eval('ID_%s' % in1_ID).query('hour==7').copy()
        except:
            exec('ID_%s = link_datetime(ID_%s)' % (in1_ID,in1_ID))
            in1_hour6 = eval('ID_%s' % in0_ID).query('hour==7').copy()
        link_hour7 = ID_link.query('hour==8').copy()

        link_hour7_inPreHourTTMean = in0_hour6[['dayofyear','travel_time']].groupby('dayofyear').agg(np.mean)
        link_hour7_inPreHourTTMean += in1_hour6[['dayofyear','travel_time']].groupby('dayofyear').agg(np.mean)
        link_hour7_inPreHourTTMean /=2
        link_hour7_inPreHourTTMean.columns=['inPreHourTTMean']
        link_hour7_inPreHourTTMean.reset_index(inplace=True)

        link_hour7_more = pd.merge(link_hour7, link_hour7_inPreHourTTMean, how='left', on='dayofyear')

    elif in_num==3:
        in0_ID = ID_link['in0'].iloc[0]
        in1_ID = ID_link['in1'].iloc[0]
        in2_ID = ID_link['in2'].iloc[0]
        try:
            in0_hour6 = eval('ID_%s' % in0_ID).query('hour==7').copy()
        except:
#             eval('ID_%s' % in0_ID) = link_datetime(eval('ID_%s' % in0_ID))
            exec('ID_%s = link_datetime(ID_%s)' % (in0_ID,in0_ID))
            in0_hour6 = eval('ID_%s' % in0_ID).query('hour==7').copy()
        try:
            in1_hour6 = eval('ID_%s' % in1_ID).query('hour==7').copy()
        except:
#             eval('ID_%s' % in1_ID) = link_datetime(eval('ID_%s' % in1_ID))
            exec('ID_%s = link_datetime(ID_%s)' % (in1_ID,in1_ID))
            in1_hour6 = eval('ID_%s' % in0_ID).query('hour==7').copy()
        try:
            in2_hour6 = eval('ID_%s' % in2_ID).query('hour==7').copy()
        except:
#             eval('ID_%s' % in2_ID) = link_datetime(eval('ID_%s' % in2_ID))
            exec('ID_%s = link_datetime(ID_%s)' % (in2_ID,in2_ID))
            in2_hour6 = eval('ID_%s' % in2_ID).query('hour==7').copy()
#         in0_hour6 = eval('ID_%s' % 'in0_ID').query('hour==6').copy()
#         in1_hour6 = eval('ID_%s' % 'in1_ID').query('hour==6').copy()
#         in2_hour6 = eval('ID_%s' % 'in2_ID').query('hour==6').copy()
        link_hour7 = ID_link.query('hour==8').copy()

        link_hour7_inPreHourTTMean = in0_hour6[['dayofyear','travel_time']].groupby('dayofyear').agg(np.mean)
        link_hour7_inPreHourTTMean += in1_hour6[['dayofyear','travel_time']].groupby('dayofyear').agg(np.mean)
        link_hour7_inPreHourTTMean += in2_hour6[['dayofyear','travel_time']].groupby('dayofyear').agg(np.mean)
        link_hour7_inPreHourTTMean /=3
        link_hour7_inPreHourTTMean.columns=['inPreHourTTMean']
        link_hour7_inPreHourTTMean.reset_index(inplace=True)

        link_hour7_more = pd.merge(link_hour7, link_hour7_inPreHourTTMean, how='left', on='dayofyear')

    elif in_num==4:
        in0_ID = ID_link['in0'].iloc[0]
        in1_ID = ID_link['in1'].iloc[0]
        in2_ID = ID_link['in2'].iloc[0]
        in3_ID = ID_link['in3'].iloc[0]
        try:
            in0_hour6 = eval('ID_%s' % in0_ID).query('hour==7').copy()
        except:
#             eval('ID_%s' % in0_ID) = link_datetime(eval('ID_%s' % in0_ID))
            exec('ID_%s = link_datetime(ID_%s)' % (in0_ID,in0_ID))
            in0_hour6 = eval('ID_%s' % in0_ID).query('hour==7').copy()
        try:
            in1_hour6 = eval('ID_%s' % in1_ID).query('hour==7').copy()
        except:
            exec('ID_%s = link_datetime(ID_%s)' % (in1_ID,in1_ID))
#             eval('ID_%s' % in1_ID) = link_datetime(eval('ID_%s' % in1_ID))
            in1_hour6 = eval('ID_%s' % in0_ID).query('hour==7').copy()
        try:
            in2_hour6 = eval('ID_%s' % in2_ID).query('hour==7').copy()
        except:
            exec('ID_%s = link_datetime(ID_%s)' % (in2_ID,in2_ID))
#             eval('ID_%s' % in2_ID) = link_datetime(eval('ID_%s' % in2_ID))
            in2_hour6 = eval('ID_%s' % in2_ID).query('hour==7').copy()
        try:
            in3_hour6 = eval('ID_%s' % in3_ID).query('hour==7').copy()
        except:
            exec('ID_%s = link_datetime(ID_%s)' % (in3_ID,in3_ID))
#             eval('ID_%s' % in3_ID) = link_datetime(eval('ID_%s' % in3_ID))
            in3_hour6 = eval('ID_%s' % in3_ID).query('hour==7').copy()
#         in0_hour6 = eval('ID_%s' % 'in0_ID').query('hour==6').copy()
#         in1_hour6 = eval('ID_%s' % 'in1_ID').query('hour==6').copy()
#         in2_hour6 = eval('ID_%s' % 'in2_ID').query('hour==6').copy()
#         in3_hour6 = eval('ID_%s' % 'in3_ID').query('hour==6').copy()
        link_hour7 = ID_link.query('hour==8').copy()

        link_hour7_inPreHourTTMean = in0_hour6[['dayofyear','travel_time']].groupby('dayofyear').agg(np.mean)
        link_hour7_inPreHourTTMean += in1_hour6[['dayofyear','travel_time']].groupby('dayofyear').agg(np.mean)
        link_hour7_inPreHourTTMean += in2_hour6[['dayofyear','travel_time']].groupby('dayofyear').agg(np.mean)
        link_hour7_inPreHourTTMean += in3_hour6[['dayofyear','travel_time']].groupby('dayofyear').agg(np.mean)
        link_hour7_inPreHourTTMean /=4
        link_hour7_inPreHourTTMean.columns=['inPreHourTTMean']
        link_hour7_inPreHourTTMean.reset_index(inplace=True)

        link_hour7_more = pd.merge(link_hour7, link_hour7_inPreHourTTMean, how='left', on='dayofyear')
    elif in_num==0:
        link_hour7_more = ID_link.query('hour==8').copy()
        link_hour7_more['inPreHourTTMean']=link_hour7_more['prehour_ttmean']
    return link_hour7_more

## 把prehour_ttmean,inPreHourTTMean 合并进入训练数据

In [17]:
for ID in link_ID_list:
    exec("tmp = prehour_ttmean(ID_%s)" % ID)
    exec("tmp1 = pd.merge(ID_%s, tmp[['minuteofyear','prehour_ttmean']], how='left',on='minuteofyear')" % ID)
    tmp2 = inPreHourTTMean(tmp1)
    exec("ID_%s_hour8 = pd.merge(tmp, tmp2[['minuteofyear','inPreHourTTMean']], how='left',on='minuteofyear')" % ID)

# 训练模型

In [18]:
def train_model(data0):
    data = data0[['hour', 'travel_time_clean', 'length', 'width',
           'link_class', 'in_num', 'out_num',  'in0_len', 'in1_len', 'in2_len', 'in3_len',
           'in0_wid', 'in1_wid', 'in2_wid', 'in3_wid', 'out0_len', 'out1_len',
           'out2_len', 'out3_len', 'out0_wid', 'out1_wid', 'out2_wid', 'out3_wid',
           'in_width', 'in_length', 'out_width', 'out_length', 'minute',
           'time_area', 'minuteofday', 'dayofweek', 'day', 'month',
           'minuteofmonth', 'minuteofyear', 'dayofyear', 'prehour_ttmean',
           'inPreHourTTMean']].copy()
    X = data[['hour', 'length', 'width',
           'link_class', 'in_num', 'out_num',  'in0_len', 'in1_len', 'in2_len', 'in3_len',
           'in0_wid', 'in1_wid', 'in2_wid', 'in3_wid', 'out0_len', 'out1_len',
           'out2_len', 'out3_len', 'out0_wid', 'out1_wid', 'out2_wid', 'out3_wid',
           'in_width', 'in_length', 'out_width', 'out_length', 'minute',
           'time_area', 'minuteofday', 'dayofweek', 'day', 'month',
           'minuteofmonth', 'minuteofyear', 'dayofyear', 'prehour_ttmean',
           'inPreHourTTMean']]
    y = data['travel_time_clean']
    X = np.array(X)
    y = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    clf = xgb.XGBRegressor()
    clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="rmse",
            eval_set=[(X_test, y_test)])
    return clf

# 训练

In [21]:
for i in link_ID_list:
    % time exec("rgr_%s = train_model(ID_%s_hour8)" % (i,i))

[0]	validation_0-rmse:6.21284
Will train until validation_0-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:5.61414
[2]	validation_0-rmse:5.07856
[3]	validation_0-rmse:4.60338
[4]	validation_0-rmse:4.17712
[5]	validation_0-rmse:3.79758
[6]	validation_0-rmse:3.46075
[7]	validation_0-rmse:3.16155
[8]	validation_0-rmse:2.89548
[9]	validation_0-rmse:2.66284
[10]	validation_0-rmse:2.45847
[11]	validation_0-rmse:2.28121
[12]	validation_0-rmse:2.12772
[13]	validation_0-rmse:1.99333
[14]	validation_0-rmse:1.88063
[15]	validation_0-rmse:1.78234
[16]	validation_0-rmse:1.69996
[17]	validation_0-rmse:1.63105
[18]	validation_0-rmse:1.57314
[19]	validation_0-rmse:1.52403
[20]	validation_0-rmse:1.48397
[21]	validation_0-rmse:1.4513
[22]	validation_0-rmse:1.4248
[23]	validation_0-rmse:1.40063
[24]	validation_0-rmse:1.38259
[25]	validation_0-rmse:1.36873
[26]	validation_0-rmse:1.3565
[27]	validation_0-rmse:1.3472
[28]	validation_0-rmse:1.33804
[29]	validation_0-rmse:1.33237
[30]	validation_0-

[56]	validation_0-rmse:27.5008
[57]	validation_0-rmse:27.4624
[58]	validation_0-rmse:27.4387
[59]	validation_0-rmse:27.3989
[60]	validation_0-rmse:27.323
[61]	validation_0-rmse:27.309
[62]	validation_0-rmse:27.2939
[63]	validation_0-rmse:27.2463
[64]	validation_0-rmse:27.238
[65]	validation_0-rmse:27.213
[66]	validation_0-rmse:27.2018
[67]	validation_0-rmse:27.169
[68]	validation_0-rmse:27.1342
[69]	validation_0-rmse:27.0746
[70]	validation_0-rmse:27.0089
[71]	validation_0-rmse:26.9855
[72]	validation_0-rmse:26.8977
[73]	validation_0-rmse:26.8909
[74]	validation_0-rmse:26.8463
[75]	validation_0-rmse:26.8332
[76]	validation_0-rmse:26.8241
[77]	validation_0-rmse:26.7819
[78]	validation_0-rmse:26.7638
[79]	validation_0-rmse:26.7586
[80]	validation_0-rmse:26.7194
[81]	validation_0-rmse:26.684
[82]	validation_0-rmse:26.68
[83]	validation_0-rmse:26.6382
[84]	validation_0-rmse:26.622
[85]	validation_0-rmse:26.5694
[86]	validation_0-rmse:26.5623
[87]	validation_0-rmse:26.5129
[88]	validation_0

[10]	validation_0-rmse:1.70895
[11]	validation_0-rmse:1.6234
[12]	validation_0-rmse:1.55185
[13]	validation_0-rmse:1.49202
[14]	validation_0-rmse:1.44152
[15]	validation_0-rmse:1.39849
[16]	validation_0-rmse:1.36124
[17]	validation_0-rmse:1.33065
[18]	validation_0-rmse:1.3049
[19]	validation_0-rmse:1.2819
[20]	validation_0-rmse:1.26406
[21]	validation_0-rmse:1.24837
[22]	validation_0-rmse:1.23523
[23]	validation_0-rmse:1.22383
[24]	validation_0-rmse:1.21569
[25]	validation_0-rmse:1.20822
[26]	validation_0-rmse:1.202
[27]	validation_0-rmse:1.19589
[28]	validation_0-rmse:1.18891
[29]	validation_0-rmse:1.18441
[30]	validation_0-rmse:1.18114
[31]	validation_0-rmse:1.17687
[32]	validation_0-rmse:1.1731
[33]	validation_0-rmse:1.17015
[34]	validation_0-rmse:1.16683
[35]	validation_0-rmse:1.16666
[36]	validation_0-rmse:1.16309
[37]	validation_0-rmse:1.16117
[38]	validation_0-rmse:1.15854
[39]	validation_0-rmse:1.15617
[40]	validation_0-rmse:1.15468
[41]	validation_0-rmse:1.1531
[42]	validation

[68]	validation_0-rmse:11.4992
[69]	validation_0-rmse:11.4932
[70]	validation_0-rmse:11.4929
[71]	validation_0-rmse:11.4913
[72]	validation_0-rmse:11.4775
[73]	validation_0-rmse:11.4679
[74]	validation_0-rmse:11.4535
[75]	validation_0-rmse:11.4399
[76]	validation_0-rmse:11.4384
[77]	validation_0-rmse:11.4304
[78]	validation_0-rmse:11.4303
[79]	validation_0-rmse:11.4201
[80]	validation_0-rmse:11.4186
[81]	validation_0-rmse:11.4089
[82]	validation_0-rmse:11.4106
[83]	validation_0-rmse:11.3904
[84]	validation_0-rmse:11.3926
[85]	validation_0-rmse:11.3851
[86]	validation_0-rmse:11.3704
[87]	validation_0-rmse:11.3779
[88]	validation_0-rmse:11.3793
[89]	validation_0-rmse:11.381
[90]	validation_0-rmse:11.3814
[91]	validation_0-rmse:11.3752
[92]	validation_0-rmse:11.3748
[93]	validation_0-rmse:11.3708
[94]	validation_0-rmse:11.3735
[95]	validation_0-rmse:11.3669
[96]	validation_0-rmse:11.3528
[97]	validation_0-rmse:11.3453
[98]	validation_0-rmse:11.3426
[99]	validation_0-rmse:11.3413
CPU times

[22]	validation_0-rmse:17.1722
[23]	validation_0-rmse:17.0858
[24]	validation_0-rmse:17.0111
[25]	validation_0-rmse:16.9469
[26]	validation_0-rmse:16.8869
[27]	validation_0-rmse:16.8362
[28]	validation_0-rmse:16.7877
[29]	validation_0-rmse:16.7516
[30]	validation_0-rmse:16.7252
[31]	validation_0-rmse:16.7063
[32]	validation_0-rmse:16.6738
[33]	validation_0-rmse:16.6536
[34]	validation_0-rmse:16.6156
[35]	validation_0-rmse:16.5888
[36]	validation_0-rmse:16.5651
[37]	validation_0-rmse:16.5615
[38]	validation_0-rmse:16.5327
[39]	validation_0-rmse:16.5129
[40]	validation_0-rmse:16.5023
[41]	validation_0-rmse:16.4924
[42]	validation_0-rmse:16.4616
[43]	validation_0-rmse:16.4503
[44]	validation_0-rmse:16.4391
[45]	validation_0-rmse:16.4139
[46]	validation_0-rmse:16.4007
[47]	validation_0-rmse:16.3753
[48]	validation_0-rmse:16.3551
[49]	validation_0-rmse:16.3368
[50]	validation_0-rmse:16.3183
[51]	validation_0-rmse:16.3059
[52]	validation_0-rmse:16.3066
[53]	validation_0-rmse:16.297
[54]	vali

...

[64]	validation_0-rmse:1.37346
[65]	validation_0-rmse:1.37226
[66]	validation_0-rmse:1.37204
[67]	validation_0-rmse:1.36907
[68]	validation_0-rmse:1.36633
[69]	validation_0-rmse:1.36549
[70]	validation_0-rmse:1.36533
[71]	validation_0-rmse:1.36478
[72]	validation_0-rmse:1.36464
[73]	validation_0-rmse:1.36384
[74]	validation_0-rmse:1.36207
[75]	validation_0-rmse:1.35981
[76]	validation_0-rmse:1.35867
[77]	validation_0-rmse:1.35838
[78]	validation_0-rmse:1.35812
[79]	validation_0-rmse:1.3582
[80]	validation_0-rmse:1.35647
[81]	validation_0-rmse:1.35629
[82]	validation_0-rmse:1.3563
[83]	validation_0-rmse:1.35606
[84]	validation_0-rmse:1.35523
[85]	validation_0-rmse:1.35494
[86]	validation_0-rmse:1.35526
[87]	validation_0-rmse:1.35433
[88]	validation_0-rmse:1.35401
[89]	validation_0-rmse:1.35243
[90]	validation_0-rmse:1.3515
[91]	validation_0-rmse:1.34984
[92]	validation_0-rmse:1.34865
[93]	validation_0-rmse:1.34798
[94]	validation_0-rmse:1.34751
[95]	validation_0-rmse:1.34685
[96]	valida

[18]	validation_0-rmse:2.86723
[19]	validation_0-rmse:2.81465
[20]	validation_0-rmse:2.75892
[21]	validation_0-rmse:2.71351
[22]	validation_0-rmse:2.67127
[23]	validation_0-rmse:2.64731
[24]	validation_0-rmse:2.61907
[25]	validation_0-rmse:2.60311
[26]	validation_0-rmse:2.57731
[27]	validation_0-rmse:2.56603
[28]	validation_0-rmse:2.54884
[29]	validation_0-rmse:2.53167
[30]	validation_0-rmse:2.5289
[31]	validation_0-rmse:2.51605
[32]	validation_0-rmse:2.50953
[33]	validation_0-rmse:2.49733
[34]	validation_0-rmse:2.49229
[35]	validation_0-rmse:2.47157
[36]	validation_0-rmse:2.4689
[37]	validation_0-rmse:2.45949
[38]	validation_0-rmse:2.4465
[39]	validation_0-rmse:2.43848
[40]	validation_0-rmse:2.43555
[41]	validation_0-rmse:2.43178
[42]	validation_0-rmse:2.41575
[43]	validation_0-rmse:2.41394
[44]	validation_0-rmse:2.407
[45]	validation_0-rmse:2.40608
[46]	validation_0-rmse:2.40316
[47]	validation_0-rmse:2.40428
[48]	validation_0-rmse:2.39398
[49]	validation_0-rmse:2.38625
[50]	validati

[76]	validation_0-rmse:13.0186
[77]	validation_0-rmse:13.0204
[78]	validation_0-rmse:13.0124
[79]	validation_0-rmse:13.0106
[80]	validation_0-rmse:12.9828
[81]	validation_0-rmse:12.9718
[82]	validation_0-rmse:12.9683
[83]	validation_0-rmse:12.9632
[84]	validation_0-rmse:12.9582
[85]	validation_0-rmse:12.9548
[86]	validation_0-rmse:12.9512
[87]	validation_0-rmse:12.9399
[88]	validation_0-rmse:12.9312
[89]	validation_0-rmse:12.9057
[90]	validation_0-rmse:12.9077
[91]	validation_0-rmse:12.9096
[92]	validation_0-rmse:12.9076
[93]	validation_0-rmse:12.8916
[94]	validation_0-rmse:12.8732
[95]	validation_0-rmse:12.8675
[96]	validation_0-rmse:12.861
[97]	validation_0-rmse:12.8313
[98]	validation_0-rmse:12.8192
[99]	validation_0-rmse:12.8029
CPU times: user 1.9 s, sys: 4 ms, total: 1.9 s
Wall time: 259 ms
[0]	validation_0-rmse:70.7899
Will train until validation_0-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:65.0902
[2]	validation_0-rmse:60.0926
[3]	validation_0-rmse:55.6804
[4]	va

[27]	validation_0-rmse:16.2368
[28]	validation_0-rmse:16.165
[29]	validation_0-rmse:16.1015
[30]	validation_0-rmse:15.9973
[31]	validation_0-rmse:15.9141
[32]	validation_0-rmse:15.8409
[33]	validation_0-rmse:15.811
[34]	validation_0-rmse:15.7506
[35]	validation_0-rmse:15.7175
[36]	validation_0-rmse:15.6779
[37]	validation_0-rmse:15.6559
[38]	validation_0-rmse:15.6074
[39]	validation_0-rmse:15.5854
[40]	validation_0-rmse:15.5613
[41]	validation_0-rmse:15.536
[42]	validation_0-rmse:15.5077
[43]	validation_0-rmse:15.4916
[44]	validation_0-rmse:15.4698
[45]	validation_0-rmse:15.4521
[46]	validation_0-rmse:15.4221
[47]	validation_0-rmse:15.3965
[48]	validation_0-rmse:15.3837
[49]	validation_0-rmse:15.3261
[50]	validation_0-rmse:15.296
[51]	validation_0-rmse:15.2589
[52]	validation_0-rmse:15.2415
[53]	validation_0-rmse:15.225
[54]	validation_0-rmse:15.2114
[55]	validation_0-rmse:15.205
[56]	validation_0-rmse:15.1656
[57]	validation_0-rmse:15.1551
[58]	validation_0-rmse:15.1314
[59]	validatio

[85]	validation_0-rmse:9.75771
[86]	validation_0-rmse:9.74412
[87]	validation_0-rmse:9.74353
[88]	validation_0-rmse:9.74086
[89]	validation_0-rmse:9.73809
[90]	validation_0-rmse:9.7389
[91]	validation_0-rmse:9.74364
[92]	validation_0-rmse:9.74038
[93]	validation_0-rmse:9.74127
[94]	validation_0-rmse:9.73686
[95]	validation_0-rmse:9.72619
[96]	validation_0-rmse:9.72279
[97]	validation_0-rmse:9.71633
[98]	validation_0-rmse:9.71371
[99]	validation_0-rmse:9.70762
CPU times: user 1.86 s, sys: 20 ms, total: 1.88 s
Wall time: 257 ms
[0]	validation_0-rmse:1.66969
Will train until validation_0-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:1.52022
[2]	validation_0-rmse:1.38747
[3]	validation_0-rmse:1.2699
[4]	validation_0-rmse:1.16603
[5]	validation_0-rmse:1.07414
[6]	validation_0-rmse:0.993248
[7]	validation_0-rmse:0.922863
[8]	validation_0-rmse:0.86189
[9]	validation_0-rmse:0.808259
[10]	validation_0-rmse:0.763015
[11]	validation_0-rmse:0.723604
[12]	validation_0-rmse:0.689697
[13]

# 制作预测特征 (ID_%s_month6_hour8)

In [22]:
for i in link_ID_list:
    exec("ID_%s_month6_7 = ID_%s.query('month==6 and hour==7').copy()" % (i,i))

In [24]:
t0=pd.date_range('2016-06-01','2016-06-30 23:58:00',freq='2min')
for i in link_ID_list:
    exec("ID_%s_month6_8 = pd.DataFrame(t0,columns=['time_interval_start'])" % i)
    exec("ID_%s_month6_8['hour']=ID_%s_month6_8['time_interval_start'].dt.hour" % (i,i))
    exec("ID_%s_month6_8 = ID_%s_month6_8.query('hour==8')" % (i,i))
    exec("ID_%s_month6_8['link_ID'] = %s" % (i,i) )
    exec("ID_%s_month6_8=pd.merge(ID_%s_month6_8, link_info_top, how='left',on='link_ID').copy()" % (i, i))
    exec("ID_%s_month6_8=link_datetime(ID_%s_month6_8)" % (i,i))

In [71]:
# tmp = pd.concat([ID_4377906284769500514_month6_7,ID_4377906284769500514_month6_8])

In [25]:
for i in link_ID_list:
    exec("ID_%s_month6_78 = pd.concat([ID_%s_month6_7, ID_%s_month6_8])" % (i,i,i))

In [26]:
for i in link_ID_list:
    exec("tmp = prehour_ttmean(ID_%s_month6_78)" % i)
    exec("tmp1 = pd.merge(ID_%s_month6_78, tmp[['minuteofyear','prehour_ttmean']], how='left',on='minuteofyear')" % i)
    tmp2 = inPreHourTTMean(tmp1)
    exec("ID_%s_month6_hour8 = pd.merge(tmp, tmp2[['minuteofyear','inPreHourTTMean']], how='left',on='minuteofyear')" % i)

#### 代码合并, 适配所有月份

In [52]:
M = 5

In [53]:

for i in link_ID_list:
    exec("ID_%s_month%d_7 = ID_%s.query('month==%d and hour==7').copy()" % (i,M,i,M))
exec("t0=pd.date_range('2016-0%d-01','2016-0%d-30 23:58:00',freq='2min')" % (M,M))
for i in link_ID_list:
    exec("ID_%s_month%d_8 = pd.DataFrame(t0,columns=['time_interval_start'])" % (i,M))
    exec("ID_%s_month%d_8['hour']=ID_%s_month%d_8['time_interval_start'].dt.hour" % (i,M,i,M))
    exec("ID_%s_month%d_8 = ID_%s_month%d_8.query('hour==8')" % (i,M,i,M))
    exec("ID_%s_month%d_8['link_ID'] = %s" % (i,M,i) )
    exec("ID_%s_month%d_8=pd.merge(ID_%s_month%d_8, link_info_top, how='left',on='link_ID').copy()" % (i, M, i,M))
    exec("ID_%s_month%d_8=link_datetime(ID_%s_month%d_8)" % (i,M,i,M))
for i in link_ID_list:
    exec("ID_%s_month%d_78 = pd.concat([ID_%s_month%d_7, ID_%s_month%d_8])" % (i,M,i,M,i,M))
for i in link_ID_list:
    exec("tmp = prehour_ttmean(ID_%s_month%d_78)" % (i,M))
    exec("tmp1 = pd.merge(ID_%s_month%d_78, tmp[['minuteofyear','prehour_ttmean']], how='left',on='minuteofyear')" % (i,M))
    tmp2 = inPreHourTTMean(tmp1)
    exec("ID_%s_month%d_hour8 = pd.merge(tmp, tmp2[['minuteofyear','inPreHourTTMean']], how='left',on='minuteofyear')" % (i,M))

## 预测6月8点到9点的数据

In [ ]:
rgr_%s

In [35]:
data0 = ID_4377906284769500514_month6_hour8
pred_features = data0[['hour', 'length', 'width',
           'link_class', 'in_num', 'out_num',  'in0_len', 'in1_len', 'in2_len', 'in3_len',
           'in0_wid', 'in1_wid', 'in2_wid', 'in3_wid', 'out0_len', 'out1_len',
           'out2_len', 'out3_len', 'out0_wid', 'out1_wid', 'out2_wid', 'out3_wid',
           'in_width', 'in_length', 'out_width', 'out_length', 'minute',
           'time_area', 'minuteofday', 'dayofweek', 'day', 'month',
           'minuteofmonth', 'minuteofyear', 'dayofyear', 'prehour_ttmean',
           'inPreHourTTMean']]
X = np.array(pred_features)

In [46]:
data0['length'].iloc[0]

11

In [37]:
exec("predictions = rgr_%s.predict(X)" % '4377906284769500514')

In [39]:
predictions.shape

(900,)

In [41]:
tt = pd.Series(predictions)

In [47]:
ans = pd.DataFrame({'time_interval_start':data0['time_interval_start'], 'travel_time':tt})
ans['travel_time'].describe()

count    900.000000
mean       0.823585
std        0.090732
min        0.646614
25%        0.757697
50%        0.811346
75%        0.881318
max        1.367102
Name: travel_time, dtype: float64

### 5月的一条link预测情况

In [62]:
data0 = ID_4377906284769500514_month5_hour8
pred_features = data0[['hour', 'length', 'width',
           'link_class', 'in_num', 'out_num',  'in0_len', 'in1_len', 'in2_len', 'in3_len',
           'in0_wid', 'in1_wid', 'in2_wid', 'in3_wid', 'out0_len', 'out1_len',
           'out2_len', 'out3_len', 'out0_wid', 'out1_wid', 'out2_wid', 'out3_wid',
           'in_width', 'in_length', 'out_width', 'out_length', 'minute',
           'time_area', 'minuteofday', 'dayofweek', 'day', 'month',
           'minuteofmonth', 'minuteofyear', 'dayofyear', 'prehour_ttmean',
           'inPreHourTTMean']]
X = np.array(pred_features)
exec("predictions = rgr_%s.predict(X)" % '4377906284769500514')
tt = pd.Series(predictions)
ans = pd.DataFrame({'time_interval_start':data0['time_interval_start'], 'travel_time_pred':tt})
ans['travel_time_pred'].describe()
ans.head()

,time_interval_start,travel_time_pred
0,2016-05-01 08:00:00,0.950097
1,2016-05-01 08:02:00,0.950097
2,2016-05-01 08:04:00,0.938887
3,2016-05-01 08:06:00,0.948059
4,2016-05-01 08:08:00,1.247396


In [65]:
ans1= pd.merge(ans, ID_4377906284769500514[['time_interval_start','travel_time']], how ='left', on='time_interval_start')

In [68]:
ans1.head()

,time_interval_start,travel_time_pred,travel_time,abs_error
0,2016-05-01 08:00:00,0.950097,0.9,0.050097
1,2016-05-01 08:02:00,0.950097,1.0,0.049903
2,2016-05-01 08:04:00,0.938887,1.0,0.061113
3,2016-05-01 08:06:00,0.948059,0.8,0.148059
4,2016-05-01 08:08:00,1.247396,5.0,3.752604


In [67]:
ans1['abs_error']=(ans1['travel_time_pred']-ans1['travel_time']).apply(np.abs)

In [69]:
ans1.describe()

,travel_time_pred,travel_time,abs_error
count,900.000000,894.000000,894.000000
mean,0.915621,1.128747,0.386840
std,0.081859,1.329791,1.265424
min,0.674278,0.400000,0.000012
25%,0.874562,0.700000,0.063216
50%,0.908723,0.900000,0.113849
75%,0.950987,1.000000,0.248884
max,1.264337,28.000000,27.030472


### 所有link的5月8点预测

In [70]:
all_link_ans=pd.DataFrame()
all_link_ans

""


In [71]:
for i in link_ID_list:
    data0 = eval("ID_%s_month5_hour8" % i)
    pred_features = data0[['hour', 'length', 'width',
               'link_class', 'in_num', 'out_num',  'in0_len', 'in1_len', 'in2_len', 'in3_len',
               'in0_wid', 'in1_wid', 'in2_wid', 'in3_wid', 'out0_len', 'out1_len',
               'out2_len', 'out3_len', 'out0_wid', 'out1_wid', 'out2_wid', 'out3_wid',
               'in_width', 'in_length', 'out_width', 'out_length', 'minute',
               'time_area', 'minuteofday', 'dayofweek', 'day', 'month',
               'minuteofmonth', 'minuteofyear', 'dayofyear', 'prehour_ttmean',
               'inPreHourTTMean']]
    X = np.array(pred_features)
    exec("predictions = rgr_%s.predict(X)" % i)
    tt = pd.Series(predictions)
    ans = pd.DataFrame({'time_interval_start':data0['time_interval_start'], 'travel_time_pred':tt})
    ans['link_ID']=i
    all_link_ans = pd.concat([all_link_ans, ans])
all_link_ans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118800 entries, 0 to 899
Data columns (total 3 columns):
time_interval_start    118800 non-null datetime64[ns]
travel_time_pred       118800 non-null float32
link_ID                118800 non-null uint64
dtypes: datetime64[ns](1), float32(1), uint64(1)
memory usage: 3.2 MB


In [73]:
all_link_ans.head()

,time_interval_start,travel_time_pred,link_ID
0,2016-05-01 08:00:00,5.972211,4377906289869500514
1,2016-05-01 08:02:00,5.972211,4377906289869500514
2,2016-05-01 08:04:00,5.921382,4377906289869500514
3,2016-05-01 08:06:00,5.921382,4377906289869500514
4,2016-05-01 08:08:00,5.951692,4377906289869500514


In [79]:
rawdata_78.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 760581 entries, 7430427 to 7599675
Data columns (total 6 columns):
link_ID                760581 non-null uint64
date                   760581 non-null object
travel_time            760581 non-null float64
time_interval_start    760581 non-null object
time_interval_end      760581 non-null object
hour                   760581 non-null int64
dtypes: float64(1), int64(1), object(3), uint64(1)
memory usage: 40.6+ MB


In [81]:
actual = rawdata_78[['link_ID','time_interval_start','travel_time']].copy()
actual['time_interval_start'] = pd.to_datetime(actual['time_interval_start'])

In [82]:
all_link_ans1 = pd.merge(all_link_ans, actual,how='left',on=['link_ID','time_interval_start'])

In [83]:
all_link_ans1.head()

,time_interval_start,travel_time_pred,link_ID,travel_time
0,2016-05-01 08:00:00,5.972211,4377906289869500514,6.6
1,2016-05-01 08:02:00,5.972211,4377906289869500514,6.6
2,2016-05-01 08:04:00,5.921382,4377906289869500514,5.5
3,2016-05-01 08:06:00,5.921382,4377906289869500514,5.5
4,2016-05-01 08:08:00,5.951692,4377906289869500514,5.5


In [84]:
all_link_ans1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118800 entries, 0 to 118799
Data columns (total 4 columns):
time_interval_start    118800 non-null datetime64[ns]
travel_time_pred       118800 non-null float32
link_ID                118800 non-null uint64
travel_time            109965 non-null float64
dtypes: datetime64[ns](1), float32(1), float64(1), uint64(1)
memory usage: 4.1 MB


In [87]:
all_link_ans1['abs_error']=(all_link_ans1['travel_time_pred']-all_link_ans1['travel_time']).apply(np.abs)
all_link_ans1['ape']=all_link_ans1['abs_error']/all_link_ans1['travel_time']

In [88]:
all_link_ans1.describe()

,travel_time_pred,link_ID,travel_time,abs_error,ape
count,118800.000000,1.188000e+05,109965.000000,109965.000000,109965.000000
mean,20.445143,4.643058e+18,24.190981,8.702458,0.330730
std,24.324057,1.471245e+18,41.042476,25.689076,0.435582
min,0.674278,3.377906e+18,0.400000,0.000012,0.000004
25%,5.217546,4.377906e+18,4.800000,0.645049,0.093273
50%,12.332829,4.377906e+18,11.500000,2.209776,0.212605
75%,25.802305,4.377906e+18,26.100000,7.350175,0.415351
max,259.718048,9.377906e+18,983.000000,858.476448,19.593585


# 可以看到5月份的8点到9点tt值预测的MAPE为0.330730

# 查看数据(以下与代码无关)

In [77]:
link_ID_list[55]

4377906285525800514

In [27]:
ID_4377906284769500514_month6_7.head()

,link_ID,date,travel_time,time_interval_start,time_interval_end,hour,travel_time_clean,length,width,link_class,...,out_length,minute,time_area,minuteofday,dayofweek,day,month,minuteofmonth,minuteofyear,dayofyear
5206,4377906284769500514,2016-06-01,1.0,2016-06-01 07:00:00,2016-06-01 07:02:00,7,1.0,11,12,1,...,15.0,0,0,420,2,1,6,1860,269700,153
5207,4377906284769500514,2016-06-01,1.0,2016-06-01 07:02:00,2016-06-01 07:04:00,7,1.0,11,12,1,...,15.0,2,1,422,2,1,6,1862,269702,153
5208,4377906284769500514,2016-06-01,1.0,2016-06-01 07:04:00,2016-06-01 07:06:00,7,1.0,11,12,1,...,15.0,4,2,424,2,1,6,1864,269704,153
5209,4377906284769500514,2016-06-01,0.7,2016-06-01 07:06:00,2016-06-01 07:08:00,7,0.7,11,12,1,...,15.0,6,3,426,2,1,6,1866,269706,153
5210,4377906284769500514,2016-06-01,0.7,2016-06-01 07:08:00,2016-06-01 07:10:00,7,0.7,11,12,1,...,15.0,8,4,428,2,1,6,1868,269708,153


In [28]:
ID_4377906285525800514_month6_78.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1751 entries, 5269 to 899
Data columns (total 49 columns):
date                   851 non-null object
day                    1751 non-null int64
dayofweek              1751 non-null int64
dayofyear              1751 non-null int64
hour                   1751 non-null int64
in0                    1751 non-null object
in0_len                1751 non-null int64
in0_wid                1751 non-null int64
in1                    0 non-null object
in1_len                1751 non-null int64
in1_wid                1751 non-null int64
in2                    0 non-null object
in2_len                1751 non-null int64
in2_wid                1751 non-null int64
in3                    0 non-null object
in3_len                1751 non-null int64
in3_wid                1751 non-null int64
in_length              1751 non-null float64
in_num                 1751 non-null int64
in_width               1751 non-null int64
length                 1751 non-nu

In [80]:
ID_4377906284769500514_month6_78.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1746 entries, 5206 to 899
Data columns (total 49 columns):
date                   846 non-null object
day                    1746 non-null int64
dayofweek              1746 non-null int64
dayofyear              1746 non-null int64
hour                   1746 non-null int64
in0                    1746 non-null object
in0_len                1746 non-null int64
in0_wid                1746 non-null int64
in1                    0 non-null object
in1_len                1746 non-null int64
in1_wid                1746 non-null int64
in2                    0 non-null object
in2_len                1746 non-null int64
in2_wid                1746 non-null int64
in3                    0 non-null object
in3_len                1746 non-null int64
in3_wid                1746 non-null int64
in_length              1746 non-null float64
in_num                 1746 non-null int64
in_width               1746 non-null int64
length                 1746 non-nu

In [29]:
ID_4377906284769500514_month6_hour8.head()

,date,day,dayofweek,dayofyear,hour,in0,in0_len,in0_wid,in1,in1_len,...,out_num,out_width,time_area,time_interval_end,time_interval_start,travel_time,travel_time_clean,width,prehour_ttmean,inPreHourTTMean
0,NaN,1,2,153,8,4377906282769500514,57,12,None,0,...,1,12,0,NaT,2016-06-01 08:00:00,NaN,NaN,12,1.36,13.296667
1,NaN,1,2,153,8,4377906282769500514,57,12,None,0,...,1,12,1,NaT,2016-06-01 08:02:00,NaN,NaN,12,1.36,13.296667
2,NaN,1,2,153,8,4377906282769500514,57,12,None,0,...,1,12,2,NaT,2016-06-01 08:04:00,NaN,NaN,12,1.36,13.296667
3,NaN,1,2,153,8,4377906282769500514,57,12,None,0,...,1,12,3,NaT,2016-06-01 08:06:00,NaN,NaN,12,1.36,13.296667
4,NaN,1,2,153,8,4377906282769500514,57,12,None,0,...,1,12,4,NaT,2016-06-01 08:08:00,NaN,NaN,12,1.36,13.296667


In [54]:
ID_4377906284769500514_month5_hour8.head()

,date,day,dayofweek,dayofyear,hour,in0,in0_len,in0_wid,in1,in1_len,...,out_num,out_width,time_area,time_interval_end,time_interval_start,travel_time,travel_time_clean,width,prehour_ttmean,inPreHourTTMean
0,NaN,1,6,122,8,4377906282769500514,57,12,None,0,...,1,12,0,NaT,2016-05-01 08:00:00,NaN,NaN,12,0.732143,8.113333
1,NaN,1,6,122,8,4377906282769500514,57,12,None,0,...,1,12,1,NaT,2016-05-01 08:02:00,NaN,NaN,12,0.732143,8.113333
2,NaN,1,6,122,8,4377906282769500514,57,12,None,0,...,1,12,2,NaT,2016-05-01 08:04:00,NaN,NaN,12,0.732143,8.113333
3,NaN,1,6,122,8,4377906282769500514,57,12,None,0,...,1,12,3,NaT,2016-05-01 08:06:00,NaN,NaN,12,0.732143,8.113333
4,NaN,1,6,122,8,4377906282769500514,57,12,None,0,...,1,12,4,NaT,2016-05-01 08:08:00,NaN,NaN,12,0.732143,8.113333


In [30]:
ID_4377906284769500514_month6_hour8.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900 entries, 0 to 899
Data columns (total 51 columns):
date                   0 non-null object
day                    900 non-null int64
dayofweek              900 non-null int64
dayofyear              900 non-null int64
hour                   900 non-null int64
in0                    900 non-null object
in0_len                900 non-null int64
in0_wid                900 non-null int64
in1                    0 non-null object
in1_len                900 non-null int64
in1_wid                900 non-null int64
in2                    0 non-null object
in2_len                900 non-null int64
in2_wid                900 non-null int64
in3                    0 non-null object
in3_len                900 non-null int64
in3_wid                900 non-null int64
in_length              900 non-null float64
in_num                 900 non-null int64
in_width               900 non-null int64
length                 900 non-null int64
link_ID       

In [24]:
ID_4377906284769500514_month6_7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 846 entries, 5206 to 6051
Data columns (total 49 columns):
link_ID                846 non-null uint64
date                   846 non-null object
travel_time            846 non-null float64
time_interval_start    846 non-null datetime64[ns]
time_interval_end      846 non-null datetime64[ns]
hour                   846 non-null int64
travel_time_clean      846 non-null float64
length                 846 non-null int64
width                  846 non-null int64
link_class             846 non-null int64
in_num                 846 non-null int64
out_num                846 non-null int64
in0                    846 non-null object
in1                    0 non-null object
in2                    0 non-null object
in3                    0 non-null object
out0                   846 non-null object
out1                   0 non-null object
out2                   0 non-null object
out3                   0 non-null object
in0_len                846 non-

In [64]:
ID_4377906284769500514_month6_8.head()

,time_interval_start,hour,link_ID,length,width,link_class,in_num,out_num,in0,in1,...,out_length,minute,time_area,minuteofday,dayofweek,day,month,minuteofmonth,minuteofyear,dayofyear
0,2016-06-01 08:00:00,8,4377906284769500514,11,12,1,1,1,4377906282769500514,None,...,15.0,0,0,480,2,1,6,1920,269760,153
1,2016-06-01 08:02:00,8,4377906284769500514,11,12,1,1,1,4377906282769500514,None,...,15.0,2,1,482,2,1,6,1922,269762,153
2,2016-06-01 08:04:00,8,4377906284769500514,11,12,1,1,1,4377906282769500514,None,...,15.0,4,2,484,2,1,6,1924,269764,153
3,2016-06-01 08:06:00,8,4377906284769500514,11,12,1,1,1,4377906282769500514,None,...,15.0,6,3,486,2,1,6,1926,269766,153
4,2016-06-01 08:08:00,8,4377906284769500514,11,12,1,1,1,4377906282769500514,None,...,15.0,8,4,488,2,1,6,1928,269768,153


In [65]:
ID_4377906284769500514_month6_8.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900 entries, 0 to 899
Data columns (total 45 columns):
time_interval_start    900 non-null datetime64[ns]
hour                   900 non-null int64
link_ID                900 non-null int64
length                 900 non-null int64
width                  900 non-null int64
link_class             900 non-null int64
in_num                 900 non-null int64
out_num                900 non-null int64
in0                    900 non-null object
in1                    0 non-null object
in2                    0 non-null object
in3                    0 non-null object
out0                   900 non-null object
out1                   0 non-null object
out2                   0 non-null object
out3                   0 non-null object
in0_len                900 non-null int64
in1_len                900 non-null int64
in2_len                900 non-null int64
in3_len                900 non-null int64
in0_wid                900 non-null int64
in1_wid 